In [1]:
import sys
import nltk
import sklearn
import pandas
import numpy

print('Python: {}'.format(sys.version))
print('NLTK: {}'.format(nltk.__version__))
print('Scikit-learn: {}'.format(sklearn.__version__))
print('Pandas: {}'.format(pandas.__version__))
print('Numpy: {}'.format(numpy.__version__))

Python: 3.6.3 (default, Oct  3 2017, 21:45:48) 
[GCC 7.2.0]
NLTK: 3.4.1
Scikit-learn: 0.21.2
Pandas: 0.24.2
Numpy: 1.16.4


In [2]:
# Carregando dados
import pandas as pd
import numpy as np

df = pd.read_csv('data/dataset_classes.txt', header=None, encoding='utf-8', sep='\t')
#df_classes_raw = pd.read_csv('data/classes_raw.txt', header=None, encoding='utf-8', sep='\t')
print(df.info())
print(df.head())

df[df.]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 793 entries, 0 to 792
Data columns (total 2 columns):
0    793 non-null object
1    793 non-null object
dtypes: object(2)
memory usage: 12.5+ KB
None
                  0                                                  1
0     mand seguranc  inexecuca contrat sanca afast reexam prov clau...
1     respons civil  acident aer decretol n dol eventual respons at...
2  processual civil  intimaca art cpc irregular nao comprov recurs ...
3     process civil  honorari execuca legitim part superior tribuna...
4         administr  servidor public artig lei n artig lei n revoga...


In [21]:
from imblearn.under_sampling import RandomUnderSampler

ros = RandomOverSampler(random_state=0)
ros.fit(X, y)
X_resampled, y_resampled = ros.sample(X, y)
colors = ['#ef8a62' if v == 0 else '#f7f7f7' if v == 1 else '#67a9cf' for v in y_resampled]
plt.scatter(X_resampled[:, 0], X_resampled[:, 1], c=colors, linewidth=0.5, edgecolor='black')
sns.despine()
plt.title("RandomOverSampler Output ($n_{class}=4700)$")

NameError: name 'RandomOverSampler' is not defined

In [19]:
# verificação das classes dos acórdãos

classes = df[0]
classes.value_counts().to_frame().style.bar()

#df_classes_raw[0].value_counts().to_frame().style.bar()

,0
processual civil,223
agrav regimental,106
tributari,68
hab corpus,59
embarg declaraca,48
process civil,47
administr,45
recurs especial,25
recurs ordinari,13
previdenciari,13


In [4]:
from sklearn.preprocessing import LabelEncoder

# convertendo classes
encoder = LabelEncoder()
classes = encoder.fit_transform(classes)

print(classes[:10])
len(classes)

[51 75 65 62  4 48 58 55 80 79]


793

In [5]:
# criação da lista de ementas
ementas = df[1]
print(ementas[:10])

0    inexecuca contrat sanca afast reexam prov clau...
1    acident aer decretol n dol eventual respons at...
2    intimaca art cpc irregular nao comprov recurs ...
3    honorari execuca legitim part superior tribuna...
4    servidor public artig lei n artig lei n revoga...
5    iss arrend mercantil incidenc mat constitucion...
6    processual penal art codig process penal conhe...
7    recurs especial art incis art ambos codig pena...
8    praz prescricional lei complement n artig lei ...
9    licitaca aca anulator indefer tutel antecip ag...
Name: 1, dtype: object


In [6]:
# uso de word2vec para a extração de features

from gensim.models import Word2Vec

ementas_list = [word for word in ementas.iteritems()]
ementas_list_list = []
for doc in ementas_list:
    ementas_list_list.append(doc[1].split())

X = Word2Vec(ementas_list_list, min_count=2)
X.wv.init_sims() # para inicializar model.vw.syn0norm, necessário para o cálculo da média da
print(X)

Word2Vec(vocab=3410, size=100, alpha=0.025)


In [7]:
# funções para calcular a média vetorial das palavras de cada ementa
import numpy as np


def word_averaging(wv, words):
    all_words, mean = set(), []
    
    for word in words:
        if isinstance(word, np.ndarray):
            mean.append(word)
        elif word in wv.vocab:
            mean.append(wv.vectors[wv.vocab[word].index])
            all_words.add(wv.vocab[word].index)

    if not mean:
        logging.warning("cannot compute similarity with no input %s", words)
        # FIXME: remove these examples in pre-processing
        return np.zeros(wv.vector_size,)

    mean = gensim.matutils.unitvec(np.array(mean).mean(axis=0)).astype(np.float32)
    return mean

def  word_averaging_list(wv, text_list):
    return np.vstack([word_averaging(wv, post) for post in text_list ])

In [8]:
# função para tokenizar as palavras com frequência maior que 2
import nltk


def w2v_tokenize_text(text):    
    tokens = []
    for sent in nltk.sent_tokenize(text, language='portuguese'):
        for word in nltk.word_tokenize(sent, language='portuguese'):
            if len(word) < 2:
                continue
            tokens.append(word)
    return tokens

In [9]:
# Like any other supervised machine learning problem, we need to divide data into 
# 20% test set and 80% training set

import gensim
from sklearn.model_selection import train_test_split
from gensim.models import Word2Vec


train, test = train_test_split(df, test_size=0.2, random_state=0)

test_tokenized = test.apply(lambda r: w2v_tokenize_text(r[1]), axis=1).values
train_tokenized = train.apply(lambda r: w2v_tokenize_text(r[1]), axis=1).values

X_train = word_averaging_list(X.wv,train_tokenized)
X_test = word_averaging_list(X.wv,test_tokenized)

y_train = train[0]
y_test = test[0]

#print(X_train)
#print(X_test)
print("train: {},{}".format(len(X_train),len(y_train)))
print("test: {},{}".format(len(X_test),len(y_test)))

train: 634,634
test: 159,159


In [12]:
# teste do uso de SMOTE
# (Synthetic Minority Over-sampling Technique) SMOTE is an 
# over-sampling method. It creates synthetic samples of the
# minority class. We use imblearn python package to 
# over-sample the minority classes.

from imblearn.over_sampling import SMOTE


smote = SMOTE('minority')

X_sm, y_sm = smote.fit_sample(X_train, y_train)
print("train: {},{}".format(len(X_sm),len(y_sm)))

ValueError: Expected n_neighbors <= n_samples,  but n_samples = 1, n_neighbors = 6

In [13]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix


# Modelos para treinar
names = ["K Nearest Neighbors", "Decision Tree", "Random Forest", "Logistic Regression", "SGD Classifier",
         "Naive Bayes", "SVM Linear"]

classifiers = [
    KNeighborsClassifier(),
    DecisionTreeClassifier(),
    RandomForestClassifier(n_estimators=1000, random_state=0),
    LogisticRegression(solver='lbfgs', multi_class='multinomial'),
    SGDClassifier(max_iter = 100, tol=1e-3),
    SVC(kernel = 'linear')
]

models = zip(names, classifiers)

for name, classifier in models:
    classifier.fit(X_train, y_train)
    y_pred = classifier.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)*100
    print("{} Accuracy: {}".format(name, accuracy))

K Nearest Neighbors Accuracy: 29.559748427672954
Decision Tree Accuracy: 26.41509433962264
Random Forest Accuracy: 35.84905660377358
Logistic Regression Accuracy: 30.81761006289308
SGD Classifier Accuracy: 37.735849056603776
Naive Bayes Accuracy: 30.81761006289308


In [14]:
# Classificador por votos
from sklearn.ensemble import VotingClassifier

names = ["K Nearest Neighbors", "Decision Tree", "Random Forest", "Logistic Regression", "SGD Classifier",
         "Naive Bayes", "SVM Linear"]

classifiers = [
    KNeighborsClassifier(),
    DecisionTreeClassifier(),
    RandomForestClassifier(n_estimators=1000, random_state=0),
    LogisticRegression(solver='lbfgs', multi_class='multinomial'),
    SGDClassifier(max_iter = 100, tol=1e-3),
    SVC(kernel = 'linear')
]

models = list(zip(names, classifiers))
zipped_models_1 = models[:]
zipped_models_2 = list(models)


votingClassifier = VotingClassifier(estimators = zipped_models_2, voting = 'hard', n_jobs = -1)
votingClassifier.fit(X_train, y_train)
y_pred = votingClassifier.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)*100
print("Voting Classifier: Accuracy: {}".format(accuracy))

Voting Classifier: Accuracy: 35.84905660377358


In [16]:
# relatório de classificação

from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

#print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))
print((accuracy_score(y_test, y_pred)*100))

                                           precision    recall  f1-score   support

                             aca rescisor       0.00      0.00      0.00         1
                                administr       0.33      0.12      0.18         8
                             agrav intern       0.00      0.00      0.00         1
                         agrav regimental       0.41      0.37      0.39        30
                  conflit negat competenc       0.00      0.00      0.00         2
                  conflit posit competenc       0.00      0.00      0.00         1
                         direit administr       0.00      0.00      0.00         1
                  direit processual civil       0.00      0.00      0.00         1
                         embarg declaraca       0.29      0.33      0.31         6
                         embarg divergenc       0.00      0.00      0.00         1
                   ement agrav regimental       0.00      0.00      0.00         1
   